# R Based Single Cell (Xena, Seurat)

This notebook demonstrates getting data from a functional genomics server (Xena), and preparing those data for analysis in Seurat.

## Acquiring Data from Xena

Xena provides an HTTP interface that accepts AST in a lisp-like syntax. Also included are some domain specific language (DSL) functions for working with functional genomics data, as well as an SQL interface.

Providing named client functions makes working with these data easier, however, for demonstration we will show how you can use the Xena query interface execute an arbitrary query.

In [56]:
# A library for sending/receiving HTTP requests
library('httr')

# The URL for the xena data we are after
hub_url <- "https://toil.xenahubs.net/data/"

# A simple query, should return 2
query <- "(+ 1 1)"

In [57]:
response <- POST(hub_url, body = query, content_type = "text/plain")
content(response)

[1] 2

### Writing Xena Queries

The Xena data model can be accessed using a lisp-like DSL. When creating a Seurat analysis, we will need a list of samples, genes, and the expressions between them.

We can also get a list of gene-names to gene-identifiers to make reading our results easier.

In [179]:
# This library allows one to perform nice string templating
library(gsubfn)

# A named dataset that contains gene-wise expression counts
dataset <- "tcga_RSEM_Hugo_norm_count"

#### Query templates

Each of these literals are a query template. Backticks are used to create logical scope for interpolating strings.

These queries will be used below to get our data. In the future, these functions could be accessed by named methods of a Xena client, as in the python client.

In [59]:
cohort_template <- '(map :cohort (query {:select [:%distinct.cohort]
                     :from [:dataset]
                     :where [:not [:is nil :cohort]]}))'

In [220]:
# fetch_template <- '(let [probemap (:probemap (car (query {:select [:probemap]
#                                                   :from [:dataset]
#                                                   :where [:= :name "`dataset`"]})))
#                  probes ((xena-query {:select ["name"] :from [probemap] :where [:in :any "genes" ["`paste(features_query, collapse = \'", "\')`"]]}) "name")]
#              [probes
#                (fetch [{:table "`dataset`"
#                         :samples ["`paste(samples_query, collapse = \'", "\')`"]
#                         :columns probes}])])'
fetch_template <- '(fetch [{:table "`dataset`"
                               :samples ["`paste(samples_query, collapse = \'", "\')`"]
                               :columns ["`paste(features_query, collapse = \'", "\')`"]}])])'

In [195]:
probemap_template <- '(:probemap (car (query {:select [:probemap]
                                      :from [:dataset]
                                      :where [:= :name "`dataset`"]})))'

In [176]:
samples_template <- '(map :value (query {:select [:value]
            :from [:dataset]
            :join [:field [:= :dataset.id :dataset_id]
            :code [:= :field.id :field_id]]
            :where [:and
            [:= :dataset.name "`dataset`"]
            [:= :field.name "sampleID"]]}))'

In [160]:
features_template <- '(map :name (query {:select [:field.name]
             :from [:dataset]
             :join [:field [:= :dataset.id :dataset_id]]
             :where [:= :dataset.name "`dataset`"]}))'

### Getting Data

#### Getting Features
First, we'll get the list of featurees for the dataset. We'll print out the query that will be sent to Xena (including newline characters).

In [180]:
query <- fn$identity(features_template)
query

[1] "(map :name (query {:select [:field.name]\n             :from [:dataset]\n             :join [:field [:= :dataset.id :dataset_id]]\n             :where [:= :dataset.name \"tcga_RSEM_Hugo_norm_count\"]}))"

*Note that we have interpolated in the dataset name to constrain our search.*

In [181]:
response <- POST(hub_url, body = query, content_type = "text/plain")

We can look at the response attributes expecting a Status 200 with some size that seems reasonable for the number of expected samples.

In [182]:
response
features = content(response)

Response [https://toil.xenahubs.net/data/]
  Date: 2017-05-10 21:44
  Status: 200
  Content-Type: application/json;charset=UTF-8
  Size: 682 kB


In [183]:
# Print out some of the features
features_table <- matrix(features, length(features))
features_table[0:10]

[[1]]
[1] "5S_rRNA"

[[2]]
[1] "5_8S_rRNA"

[[3]]
[1] "7SK"

[[4]]
[1] "A1BG"

[[5]]
[1] "A1BG-AS1"

[[6]]
[1] "A1CF"

[[7]]
[1] "A2M"

[[8]]
[1] "A2M-AS1"

[[9]]
[1] "A2ML1"

[[10]]
[1] "A2ML1-AS1"

We then write the table to file so that it can be used by Seurat (and others). 

In [184]:
write.table(features_table, 'genes.tsv', sep = '\t', append=F, quote=F, col.names=F, row.names=F)

#### Getting Samples

Now we will query the server for the available samples in the dataset.

In [185]:
query <- fn$identity(samples_template)
query
response <- POST(hub_url, body = query, content_type = "text/plain")
response
samples = content(response)
samples_table = matrix(samples, length(samples))
samples_table[0:10]

[1] "(map :value (query {:select [:value]\n            :from [:dataset]\n            :join [:field [:= :dataset.id :dataset_id]\n            :code [:= :field.id :field_id]]\n            :where [:and\n            [:= :dataset.name \"tcga_RSEM_Hugo_norm_count\"]\n            [:= :field.name \"sampleID\"]]}))"

Response [https://toil.xenahubs.net/data/]
  Date: 2017-05-10 21:44
  Status: 200
  Content-Type: application/json;charset=UTF-8
  Size: 190 kB


[[1]]
[1] "TCGA-OR-A5JX-01"

[[2]]
[1] "TCGA-HV-A5A5-01"

[[3]]
[1] "TCGA-MQ-A4LP-01"

[[4]]
[1] "TCGA-FD-A3N5-01"

[[5]]
[1] "TCGA-EP-A2KC-01"

[[6]]
[1] "TCGA-BH-A1F0-11"

[[7]]
[1] "TCGA-A8-A079-01"

[[8]]
[1] "TCGA-EL-A4JZ-01"

[[9]]
[1] "TCGA-LH-A9QB-06"

[[10]]
[1] "TCGA-DK-A2I2-01"

And then write the table to file in a similar fashion to the gene list.

In [186]:
write.table(samples_table, 'barcodes.tsv', sep = '\t', append=F, quote=F, col.names=F, row.names=F)

#### Getting Expression Data

Now that we have the list of samples and genes quantified, we can select all, or a subset of the samples from Xena.

In [221]:
features_query <- features_table[9000:10000]
samples_query <- samples_table[0:10]
query <- fn$identity(fetch_template)
query

[1] "(fetch [{:table \"tcga_RSEM_Hugo_norm_count\"\n                               :samples [\"TCGA-OR-A5JX-01\", \"TCGA-HV-A5A5-01\", \"TCGA-MQ-A4LP-01\", \"TCGA-FD-A3N5-01\", \"TCGA-EP-A2KC-01\", \"TCGA-BH-A1F0-11\", \"TCGA-A8-A079-01\", \"TCGA-EL-A4JZ-01\", \"TCGA-LH-A9QB-06\", \"TCGA-DK-A2I2-01\"]\n                               :columns [\"CCDC157\", \"CCDC158\", \"CCDC159\", \"CCDC160\", \"CCDC162P\", \"CCDC163P\", \"CCDC166\", \"CCDC167\", \"CCDC168\", \"CCDC169\", \"CCDC169-SOHLH2\", \"CCDC17\", \"CCDC170\", \"CCDC171\", \"CCDC172\", \"CCDC173\", \"CCDC174\", \"CCDC175\", \"CCDC176\", \"CCDC177\", \"CCDC178\", \"CCDC179\", \"CCDC18\", \"CCDC180\", \"CCDC181\", \"CCDC182\", \"CCDC183\", \"CCDC183-AS1\", \"CCDC184\", \"CCDC185\", \"CCDC186\", \"CCDC22\", \"CCDC23\", \"CCDC24\", \"CCDC25\", \"CCDC26\", \"CCDC27\", \"CCDC28A\", \"CCDC28B\", \"CCDC3\", \"CCDC30\", \"CCDC33\", \"CCDC34\", \"CCDC34P1\", \"CCDC36\", \"CCDC37\", \"CCDC37-AS1\", \"CCDC38\", \"CCDC39\", \"CCDC39-AS1\", \"CCDC40\", \"CCDC42\", \"CCDC42B\", \"CCDC43\", \"CCDC47\", \"CCDC50\", \"CCDC51\", \"CCDC53\", \"CCDC54\", \"CCDC57\", \"CCDC58\", \"CCDC58P1\", \"CCDC58P2\", \"CCDC58P3\", \"CCDC58P5\", \"CCDC59\", \"CCDC6\", \"CCDC60\", \"CCDC61\", \"CCDC62\", \"CCDC63\", \"CCDC64\", \"CCDC64B\", \"CCDC65\", \"CCDC66\", \"CCDC67\", \"CCDC68\", \"CCDC69\", \"CCDC7\", \"CCDC70\", \"CCDC71\", \"CCDC71L\", \"CCDC73\", \"CCDC74A\", \"CCDC74B\", \"CCDC75P1\", \"CCDC77\", \"CCDC78\", \"CCDC79\", \"CCDC8\", \"CCDC80\", \"CCDC81\", \"CCDC82\", \"CCDC83\", \"CCDC84\", \"CCDC85A\", \"CCDC85B\", \"CCDC85C\", \"CCDC86\", \"CCDC87\", \"CCDC88A\", \"CCDC88B\", \"CCDC88C\", \"CCDC89\", \"CCDC9\", \"CCDC90B\", \"CCDC91\", \"CCDC92\", \"CCDC93\", \"CCDC94\", \"CCDC96\", \"CCDC97\", \"CCER1\", \"CCER2\", \"CCHCR1\", \"CCIN\", \"CCK\", \"CCKAR\", \"CCKBR\", \"CCL1\", \"CCL11\", \"CCL13\", \"CCL14\", \"CCL15\", \"CCL15-CCL14\", \"CCL16\", \"CCL17\", \"CCL18\", \"CCL19\", \"CCL2\", \"CCL20\", \"CCL21\", \"CCL22\", \"CCL23\", \"CCL24\", \"CCL25\", \"CCL26\", \"CCL27\", \"CCL28\", \"CCL3\", \"CCL3L3\", \"CCL4\", \"CCL4L2\", \"CCL5\", \"CCL7\", \"CCL8\", \"CCM2\", \"CCM2L\", \"CCNA1\", \"CCNA2\", \"CCNB1\", \"CCNB1IP1\", \"CCNB1IP1P1\", \"CCNB1IP1P2\", \"CCNB1IP1P3\", \"CCNB2\", \"CCNB2P1\", \"CCNB3\", \"CCNB3P1\", \"CCNC\", \"CCND1\", \"CCND2\", \"CCND2-AS1\", \"CCND2-AS2\", \"CCND2P1\", \"CCND3\", \"CCND3P1\", \"CCND3P2\", \"CCNDBP1\", \"CCNE1\", \"CCNE2\", \"CCNF\", \"CCNG1\", \"CCNG1P1\", \"CCNG2\", \"CCNG2P1\", \"CCNH\", \"CCNHP1\", \"CCNI\", \"CCNI2\", \"CCNJ\", \"CCNJL\", \"CCNJP1\", \"CCNJP2\", \"CCNK\", \"CCNL1\", \"CCNL2\", \"CCNL2P1\", \"CCNO\", \"CCNT1\", \"CCNT2\", \"CCNT2-AS1\", \"CCNT2P1\", \"CCNY\", \"CCNYL1\", \"CCNYL2\", \"CCNYL3\", \"CCP110\", \"CCPG1\", \"CCR1\", \"CCR10\", \"CCR12P\", \"CCR2\", \"CCR3\", \"CCR4\", \"CCR5\", \"CCR6\", \"CCR7\", \"CCR8\", \"CCR9\", \"CCRL2\", \"CCRN4L\", \"CCS\", \"CCSAP\", \"CCSER1\", \"CCSER2\", \"CCT2\", \"CCT3\", \"CCT4\", \"CCT4P1\", \"CCT4P2\", \"CCT5\", \"CCT5P1\", \"CCT5P2\", \"CCT6A\", \"CCT6B\", \"CCT6P1\", \"CCT6P2\", \"CCT6P3\", \"CCT6P4\", \"CCT7\", \"CCT7P1\", \"CCT7P2\", \"CCT8\", \"CCT8L1P\", \"CCT8L2\", \"CCT8P1\", \"CCZ1\", \"CCZ1B\", \"CD101\", \"CD109\", \"CD14\", \"CD151\", \"CD160\", \"CD163\", \"CD163L1\", \"CD164\", \"CD164L2\", \"CD177\", \"CD177P1\", \"CD180\", \"CD19\", \"CD1A\", \"CD1B\", \"CD1C\", \"CD1D\", \"CD1E\", \"CD2\", \"CD200\", \"CD200R1\", \"CD200R1L\", \"CD207\", \"CD209\", \"CD22\", \"CD226\", \"CD24\", \"CD244\", \"CD247\", \"CD248\", \"CD24P2\", \"CD24P4\", \"CD27\", \"CD27-AS1\", \"CD274\", \"CD276\", \"CD28\", \"CD2AP\", \"CD2BP2\", \"CD300A\", \"CD300C\", \"CD300E\", \"CD300LB\", \"CD300LD\", \"CD300LF\", \"CD300LG\", \"CD302\", \"CD320\", \"CD33\", \"CD34\", \"CD36\", \"CD37\", \"CD38\", \"CD3D\", \"CD3E\", \"CD3EAP\", \"CD3G\", \"CD4\", \"CD40\", \"CD40LG\", \"CD44\", \"CD46\", \"CD46P1\", \"CD47\", \"CD48\", \"CD5\", \"CD52\", \"CD53\", \"CD55\", \"CD58\", \"CD59\", \"CD5L\", \"CD6\", 

Now, with a fully formed query, we can request the weights.

In [222]:
response <- POST(hub_url, body = query, content_type = "text/plain")
response
weights = content(response)
weights_table = matrix(weights, length(weights))
weights_table

Response [https://toil.xenahubs.net/data/]
  Date: 2017-05-10 23:45
  Status: 200
  Content-Type: application/json;charset=UTF-8
  Size: 82.5 kB


"6.3605, 7.2454, 6.4133, 6.5235, 5.9171, 6.5314, 7.9468, 7.6677, 6.8897, 6.8016"
"2.3196, 3.1110, 0.8453, 0.6406, 5.1923, 5.9210, 4.2269, 4.0466, 2.5145, 1.3055"
"10.6845, 8.7668, 7.4487, 7.2876, 10.2337, 8.3744, 8.3029, 9.6237, 8.9718, 8.1085"
"3.3193, 5.2612, 0.0000, 0.0000, 0.0000, 6.1882, 5.6105, 1.1337, 0.7428, 0.0000"
"3.3193, 5.9371, 4.9667, 1.6942, 0.8849, 5.0822, 3.5404, 2.3729, 2.3335, 4.3314"
"4.9041, 4.5280, 6.3897, 5.9394, 6.2707, 7.0443, 7.4777, 5.1059, 5.8805, 6.9951"
"0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6698, 0.0000, 0.0000, 0.7956"
"8.9130, 7.8990, 9.4405, 8.7454, 10.5108, 7.2236, 9.5579, 7.8915, 9.5837, 8.8712"
"3.1674, 1.8265, 3.0302, 2.7202, 2.1332, 1.0196, 2.6224, 3.3422, 1.5946, 2.2263"
"4.5208, 2.6072, 5.1719, 3.8985, 5.0148, 5.1247, 3.3859, 3.9935, 3.2860, 4.7181"
"0.0000, 0.0000, 0.0000, 1.1855, 0.0000, 1.8843, 0.0000, 0.0000, 1.5946, 2.0783"


In [214]:
weights_table

<0 x 0 matrix>